In [ ]:
import json
import math
import os
import random
import shutil
import sys
from collections import Counter, OrderedDict
from pathlib import Path
from typing import ClassVar, Iterator, Sequence

import numpy as np
import pandas as pd
import scipy.stats
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import DataLoader, Dataset, Sampler, Subset
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import StratifiedKFold
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, matthews_corrcoef

In [ ]:
df_train = pd.read_csv("/content/train.csv")
df_dev = pd.read_csv("/content/dev.csv")
df_test = pd.read_csv("/content/test.csv")

In [ ]:
df_train

,sequence,label
0,CAGGCAAATCAACTTATTTAAATTTCTTGAAATCACAATTAGGATT...,21
1,GTGAAATTGTATATGTGGATCATCAATTTTCCAATATTTTTATAAA...,20
2,GCACACCGCACGACCGTACGCGCGACCGCTACTCATCACGACGCAT...,14
3,TAAGGGAAAGAAAAACCTTTTTCTATAATATAAAACCAATTATTTA...,10
4,CATTAATAGAACCATCCAATTTAGACAAATAGTATACGGGACTTTG...,20
...,...,...
3995,AGATATTATTATACGAAATATGCCAAAAGATTATGATAAATCCATA...,20
3996,GATGATCTTCTTTCTAGATATGCAACATATTTTCCTGACATTATTG...,19
3997,TATATGAAGAAAATTGATGTTTTTGTCAAAGCTAATTTCTTTGAAG...,0
3998,AAATATAGTATAATAAATTTATCTGATAGAATCAAATCTGCTTCCA...,20


In [ ]:
len(df_train["sequence"][0])

5000

In [ ]:
batch_size = 32

In [ ]:
encoding_map = {
    "A": [1, 0, 0, 0],
    "T": [0, 1, 0, 0],
    "C": [0, 0, 1, 0],
    "G": [0, 0, 0, 1]
}

In [ ]:
def encode_sequence(sequence):
    encoded_seq = [encoding_map[base] for base in sequence if base in encoding_map]
    return torch.tensor(encoded_seq, dtype=torch.float).t()

def pad_sequences(sequences, target_length):
    padded_sequences = []
    for seq in sequences:
        seq_len = seq.size(1)
        if seq_len < target_length:
            left_padding = (target_length - seq_len) // 2
            right_padding = target_length - seq_len - left_padding
            padded_seq = torch.nn.functional.pad(seq, (left_padding, right_padding))
        else:
            padded_seq = seq[:, :target_length]
        padded_sequences.append(padded_seq)
    return torch.stack(padded_sequences)

def collate_fn(batch):
    sequences, labels = zip(*batch)
    sequence_lengths = [seq.size(1) for seq in sequences]
    target_length = int(np.median(sequence_lengths))
    padded_sequences = pad_sequences(sequences, target_length)
    labels = torch.stack(labels)
    return padded_sequences, labels

In [ ]:
class DNASequencesDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.data = self.data[~self.data['sequence'].str.contains('N')].reset_index(drop=True)
        self.encoded_sequences = [encode_sequence(seq) for seq in self.data['sequence']]
        self.labels = torch.tensor(self.data['label'].values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.encoded_sequences[idx], self.labels[idx]

In [ ]:
train_dataset = DNASequencesDataset(df_train)
dev_dataset = DNASequencesDataset(df_dev)
test_dataset = DNASequencesDataset(df_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

#Model

KAN Linear

In [ ]:
import torch
import torch.nn.functional as F
import math


class KANLinear(torch.nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        enable_standalone_scale_spline=True,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = (
            (
                torch.arange(-spline_order, grid_size + spline_order + 1) * h
                + grid_range[0]
            )
            .expand(in_features, -1)
            .contiguous()
        )
        self.register_buffer("grid", grid)

        self.base_weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        self.spline_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = torch.nn.Parameter(
                torch.Tensor(out_features, in_features)
            )

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = (
                (
                    torch.rand(self.grid_size + 1, self.in_features, self.out_features)
                    - 1 / 2
                )
                * self.scale_noise
                / self.grid_size
            )
            self.spline_weight.data.copy_(
                (self.scale_spline if not self.enable_standalone_scale_spline else 1.0)
                * self.curve2coeff(
                    self.grid.T[self.spline_order : -self.spline_order],
                    noise,
                )
            )
            if self.enable_standalone_scale_spline:
                # torch.nn.init.constant_(self.spline_scaler, self.scale_spline)
                torch.nn.init.kaiming_uniform_(self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x: torch.Tensor):
        """
        Compute the B-spline bases for the given input tensor.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).

        Returns:
            torch.Tensor: B-spline bases tensor of shape (batch_size, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features

        grid: torch.Tensor = (
            self.grid
        )  # (in_features, grid_size + 2 * spline_order + 1)
        x = x.unsqueeze(-1)
        bases = ((x >= grid[:, :-1]) & (x < grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - grid[:, : -(k + 1)])
                / (grid[:, k:-1] - grid[:, : -(k + 1)])
                * bases[:, :, :-1]
            ) + (
                (grid[:, k + 1 :] - x)
                / (grid[:, k + 1 :] - grid[:, 1:(-k)])
                * bases[:, :, 1:]
            )

        assert bases.size() == (
            x.size(0),
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return bases.contiguous()

    def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
        """
        Compute the coefficients of the curve that interpolates the given points.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).
            y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

        Returns:
            torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features
        assert y.size() == (x.size(0), self.in_features, self.out_features)

        A = self.b_splines(x).transpose(
            0, 1
        )  # (in_features, batch_size, grid_size + spline_order)
        B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
        solution = torch.linalg.lstsq(
            A, B
        ).solution  # (in_features, grid_size + spline_order, out_features)
        result = solution.permute(
            2, 0, 1
        )  # (out_features, in_features, grid_size + spline_order)

        assert result.size() == (
            self.out_features,
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return result.contiguous()

    @property
    def scaled_spline_weight(self):
        return self.spline_weight * (
            self.spline_scaler.unsqueeze(-1)
            if self.enable_standalone_scale_spline
            else 1.0
        )

    def forward(self, x: torch.Tensor):
        assert x.size(-1) == self.in_features
        original_shape = x.shape
        x = x.reshape(-1, self.in_features)

        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1),
            self.scaled_spline_weight.view(self.out_features, -1),
        )
        output = base_output + spline_output

        output = output.reshape(*original_shape[:-1], self.out_features)
        return output

    @torch.no_grad()
    def update_grid(self, x: torch.Tensor, margin=0.01):
        assert x.dim() == 2 and x.size(1) == self.in_features
        batch = x.size(0)

        splines = self.b_splines(x)  # (batch, in, coeff)
        splines = splines.permute(1, 0, 2)  # (in, batch, coeff)
        orig_coeff = self.scaled_spline_weight  # (out, in, coeff)
        orig_coeff = orig_coeff.permute(1, 2, 0)  # (in, coeff, out)
        unreduced_spline_output = torch.bmm(splines, orig_coeff)  # (in, batch, out)
        unreduced_spline_output = unreduced_spline_output.permute(
            1, 0, 2
        )  # (batch, in, out)

        # sort each channel individually to collect data distribution
        x_sorted = torch.sort(x, dim=0)[0]
        grid_adaptive = x_sorted[
            torch.linspace(
                0, batch - 1, self.grid_size + 1, dtype=torch.int64, device=x.device
            )
        ]

        uniform_step = (x_sorted[-1] - x_sorted[0] + 2 * margin) / self.grid_size
        grid_uniform = (
            torch.arange(
                self.grid_size + 1, dtype=torch.float32, device=x.device
            ).unsqueeze(1)
            * uniform_step
            + x_sorted[0]
            - margin
        )

        grid = self.grid_eps * grid_uniform + (1 - self.grid_eps) * grid_adaptive
        grid = torch.concatenate(
            [
                grid[:1]
                - uniform_step
                * torch.arange(self.spline_order, 0, -1, device=x.device).unsqueeze(1),
                grid,
                grid[-1:]
                + uniform_step
                * torch.arange(1, self.spline_order + 1, device=x.device).unsqueeze(1),
            ],
            dim=0,
        )

        self.grid.copy_(grid.T)
        self.spline_weight.data.copy_(self.curve2coeff(x, unreduced_spline_output))

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        """
        Compute the regularization loss.

        This is a dumb simulation of the original L1 regularization as stated in the
        paper, since the original one requires computing absolutes and entropy from the
        expanded (batch, in_features, out_features) intermediate tensor, which is hidden
        behind the F.linear function if we want an memory efficient implementation.

        The L1 regularization is now computed as mean absolute value of the spline
        weights. The authors implementation also includes this term in addition to the
        sample-based regularization.
        """
        l1_fake = self.spline_weight.abs().mean(-1)
        regularization_loss_activation = l1_fake.sum()
        p = l1_fake / regularization_loss_activation
        regularization_loss_entropy = -torch.sum(p * p.log())
        return (
            regularize_activation * regularization_loss_activation
            + regularize_entropy * regularization_loss_entropy
        )


class KAN(torch.nn.Module):
    def __init__(
        self,
        layers_hidden,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KAN, self).__init__()
        self.grid_size = grid_size
        self.spline_order = spline_order

        self.layers = torch.nn.ModuleList()
        for in_features, out_features in zip(layers_hidden, layers_hidden[1:]):
            self.layers.append(
                KANLinear(
                    in_features,
                    out_features,
                    grid_size=grid_size,
                    spline_order=spline_order,
                    scale_noise=scale_noise,
                    scale_base=scale_base,
                    scale_spline=scale_spline,
                    base_activation=base_activation,
                    grid_eps=grid_eps,
                    grid_range=grid_range,
                )
            )

    def forward(self, x: torch.Tensor, update_grid=True):
        for layer in self.layers:
            if update_grid:
                layer.update_grid(x)
            x = layer(x)
        return x

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        return sum(
            layer.regularization_loss(regularize_activation, regularize_entropy)
            for layer in self.layers
        )

In [ ]:
class Bilinear(nn.Module):
    """
    Simplified Bilinear layer that uses a single Linear layer for pairwise interaction.
    """
    def __init__(self, n: int, out=None, bias=False):
        super().__init__()
        if out is None:
            out = n
        # self.fc = nn.Linear(n, out, bias=bias)
        self.fc = KANLinear(n, out)

    def forward(self, x):
        # Flatten input if necessary
        return self.fc(x)

# Simplified SELayer without TRL and Concater
class SELayer(nn.Module):
    """
    Simplified Squeeze-and-Excite layer.

    Parameters
    ----------
    inp : int
        Middle layer size.
    oup : int
        Input and output size.
    reduction : int, optional
        Reduction parameter. Default is 4.
    """
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            KANLinear(oup, inp // reduction),
            # nn.Linear(oup, inp // reduction, bias=False),
            # nn.SiLU(),
            KANLinear(inp // reduction, oup),
            # nn.Linear(inp // reduction, oup, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = x.view(b, c, -1).mean(dim=2)  # Global Average Pooling
        y = self.fc(y).view(b, c, 1)
        return x * y

# Main SeqNN model
class SeqNN(nn.Module):
    """
    LegNet neural network for binary classification.
    """
    __constants__ = ('resize_factor')

    def __init__(self,
                seqsize,
                use_single_channel,
                use_reverse_channel,
                use_multisubstate_channel,
                block_sizes=[256, 256, 128, 128, 64, 64, 32, 32],
                ks=5,
                resize_factor=4,
                activation=nn.SiLU,
                filter_per_group=2,
                se_reduction=4,
                final_ch=1,
                bn_momentum=0.1):
        super().__init__()
        self.block_sizes = block_sizes
        self.resize_factor = resize_factor
        self.se_reduction = se_reduction
        self.seqsize = seqsize
        self.use_single_channel = use_single_channel
        self.use_reverse_channel = use_reverse_channel
        self.use_multisubstate_channel = use_multisubstate_channel
        self.final_ch = final_ch
        self.bn_momentum = bn_momentum
        seqextblocks = OrderedDict()

        in_channels_first_block = 4
        if self.use_single_channel:
            in_channels_first_block += 1
        if self.use_reverse_channel:
            in_channels_first_block += 1
        if self.use_multisubstate_channel:
            in_channels_first_block += 1

        block = nn.Sequential(
            nn.Conv1d(
                in_channels=in_channels_first_block,
                out_channels=block_sizes[0],
                kernel_size=ks,
                padding='same',
                bias=False
            ),
            nn.BatchNorm1d(block_sizes[0], momentum=self.bn_momentum),
            activation()
        )
        seqextblocks['blc0'] = block

        # Building remaining blocks
        for ind, (prev_sz, sz) in enumerate(zip(block_sizes[:-1], block_sizes[1:])):
            block = nn.Sequential(
                nn.Conv1d(prev_sz, sz * self.resize_factor, kernel_size=1, padding='same', bias=False),
                nn.BatchNorm1d(sz * self.resize_factor, momentum=self.bn_momentum),
                activation(),

                nn.Conv1d(sz * self.resize_factor, sz * self.resize_factor, kernel_size=ks,
                          groups=sz * self.resize_factor // filter_per_group, padding='same', bias=False),
                nn.BatchNorm1d(sz * self.resize_factor, momentum=self.bn_momentum),
                activation(),

                SELayer(prev_sz, sz * self.resize_factor, reduction=self.se_reduction),

                nn.Conv1d(sz * self.resize_factor, prev_sz, kernel_size=1, padding='same', bias=False),
                nn.BatchNorm1d(prev_sz, momentum=self.bn_momentum),
                activation(),
            )
            seqextblocks[f'inv_res_blc{ind}'] = block

            resize_block = nn.Sequential(
                nn.Conv1d(2 * prev_sz, sz, kernel_size=ks, padding='same', bias=False),
                nn.BatchNorm1d(sz, momentum=self.bn_momentum),
                activation()
            )
            seqextblocks[f'resize_blc{ind}'] = resize_block

        self.seqextractor = nn.ModuleDict(seqextblocks)

        self.mapper = nn.Sequential(
            nn.Conv1d(block_sizes[-1], self.final_ch, kernel_size=1, padding='same'),
            activation()
        )

        self.register_buffer('bins', torch.arange(start=0, end=self.final_ch, step=1, requires_grad=False))

    def feature_extractor(self, x):
        x = self.seqextractor['blc0'](x)
        for i in range(len(self.block_sizes) - 1):
            x = torch.cat([x, self.seqextractor[f'inv_res_blc{i}'](x)], dim=1)
            x = self.seqextractor[f'resize_blc{i}'](x)
        return x

    def forward(self, x):
        f = self.feature_extractor(x)
        x = self.mapper(f)
        x = F.adaptive_avg_pool1d(x, 1)
        x = x.squeeze(2)
        prob = torch.sigmoid(x).squeeze(1)

        return prob
        # logprobs = F.log_softmax(x, dim=1)

        # # Soft-argmax operation (optional)
        # x = F.softmax(x, dim=1)
        # score = (x * self.bins).sum(dim=1)

        # return logprobs, score


KAN Conv

In [ ]:
#Util
def add_padding_1d(array: np.ndarray, padding: int) -> np.ndarray:
    """Adds padding to a 1D array."""
    n = array.shape[0]
    padded_array = np.zeros(n + 2 * padding)
    padded_array[padding: n + padding] = array
    return padded_array


def calc_out_dims_1d(array, kernel_size, stride, dilation, padding):
    """Calculate output dimensions for 1D convolution."""
    batch_size, n_channels, n = matrix.shape
    out_size = np.floor((n + 2 * padding - kernel_size - (kernel_size - 1) * (dilation - 1)) / stride).astype(int) + 1
    return out_size, batch_size, n_channels


def multiple_convs_kan_conv1d(array,
                               kernels,
                               kernel_size,
                               out_channels,
                               stride=1,
                               dilation=1,
                               padding=0,
                               device="cuda") -> torch.Tensor:
    """Performs a 1D convolution with multiple kernels on the input array using specified stride, dilation, and padding.

    Args:
        array (torch.Tensor): 1D tensor of shape (batch_size, channels, length).
        kernels (list): List of kernel functions to be applied.
        kernel_size (int): Size of the 1D kernel.
        out_channels (int): Number of output channels.
        stride (int): Stride along the length of the array. Default is 1.
        dilation (int): Dilation rate along the length of the array. Default is 1.
        padding (int): Number of elements to pad on each side. Default is 0.
        device (str): Device to perform calculations on. Default is "cuda".

    Returns:
        torch.Tensor: Feature map after convolution with shape (batch_size, out_channels, length_out).
    """
    length_out, batch_size = calc_out_dims_1d(array, kernel_size, stride, dilation, padding)
    n_convs = len(kernels)

    array_out = torch.zeros((batch_size, out_channels, length_out)).to(device)

    array = F.pad(array, (padding, padding), mode='constant', value=0)
    conv_groups = array.unfold(2, kernel_size, stride)
    conv_groups = conv_groups.contiguous()

    kern_per_out = len(kernels) // out_channels

    for c_out in range(out_channels):
        out_channel_accum = torch.zeros((batch_size, length_out), device=device)

        for k_idx in range(kern_per_out):
            kernel = kernels[c_out * kern_per_out + k_idx]
            conv_result = kernel(conv_groups.view(-1, 1, kernel_size))
            out_channel_accum += conv_result.view(batch_size, length_out)

        array_out[:, c_out, :] = out_channel_accum

    return array_out

In [ ]:
def kan_conv1d(matrix: torch.Tensor,
               kernel,
               kernel_size: int,
               stride: int = 1,
               dilation: int = 1,
               padding: int = 0,
               device: str = "cpu") -> torch.Tensor:
    """
    Performs a 1D convolution with the given kernel over a 1D matrix using the defined stride, dilation, and padding.

    Args:
        matrix (torch.Tensor): 3D tensor (batch_size, channels, width) to be convolved.
        kernel (function): Kernel function to apply on the 1D patches of the matrix.
        kernel_size (int): Size of the kernel (assumed to be square).
        stride (int, optional): Stride along the width axis. Defaults to 1.
        dilation (int, optional): Dilation along the width axis. Defaults to 1.
        padding (int, optional): Padding along the width axis. Defaults to 0.
        device (str): Device to perform the operation on (e.g., "cuda" or "cpu").

    Returns:
        torch.Tensor: 1D Feature map after convolution.
    """

    batch_size, n_channels, width_in = matrix.shape
    width_out = ((width_in + 2 * padding - dilation * (kernel_size - 1) - 1) // stride) + 1
    matrix_out = torch.zeros((batch_size, n_channels, width_out), device=device)

    matrix_padded = torch.nn.functional.pad(matrix, (padding, padding))

    for i in range(width_out):

        start = i * stride
        end = start + kernel_size * dilation
        patch = matrix_padded[:, :, start:end:dilation]

        matrix_out[:, :, i] = kernel.forward(patch).squeeze(-1)

    return matrix_out

In [ ]:
class KAN_Convolutional_Layer_1D(torch.nn.Module):
    def __init__(self, in_channels=1, out_channels=1, kernel_size=5, stride=1, padding=0, dilation=1, device="cuda"):
        super(KAN_Convolutional_Layer_1D, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.device = device
        self.convs = torch.nn.ModuleList([KAN_Convolution_1D(kernel_size, stride, padding, dilation, device) for _ in range(in_channels * out_channels)])

    def forward(self, x: torch.Tensor):
        return torch.cat([conv(x[:, i, :].unsqueeze(1)) for i, conv in enumerate(self.convs)], dim=1)

In [ ]:
class KAN_Convolutional_Layer_1D(torch.nn.Module):
    def __init__(
            self,
            in_channels: int = 1,
            out_channels: int = 1,
            kernel_size: int = 2,
            stride: int = 1,
            padding: int = 0,
            dilation: int = 1,
            grid_size: int = 5,
            spline_order: int = 3,
            scale_noise: float = 0.1,
            scale_base: float = 1.0,
            scale_spline: float = 1.0,
            base_activation=torch.nn.SiLU,
            grid_eps: float = 0.02,
            grid_range: tuple = [-1, 1],
            device: str = "cpu"
        ):
        super(KAN_Convolutional_Layer_1D, self).__init__()
        self.out_channels = out_channels
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.padding = padding
        self.stride = stride


        self.convs = torch.nn.ModuleList()
        for _ in range(in_channels * out_channels):
            self.convs.append(
                KAN_Convolution_1D(
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    dilation=dilation,
                    grid_size=grid_size,
                    spline_order=spline_order,
                    scale_noise=scale_noise,
                    scale_base=scale_base,
                    scale_spline=scale_spline,
                    base_activation=base_activation,
                    grid_eps=grid_eps,
                    grid_range=grid_range,
                )
            )

    def forward(self, x: torch.Tensor):
        batch_size, in_channels, length = x.shape
        output_length = (length + 2 * self.padding - self.dilation * (self.kernel_size - 1) - 1) // self.stride + 1
        output = torch.zeros((batch_size, self.out_channels, output_length), device=x.device)


        for i in range(self.out_channels):
            output_accum = torch.zeros((batch_size, output_length), device=x.device)
            for j in range(self.in_channels):
                kernel_idx = i * self.in_channels + j
                conv_result = self.convs[kernel_idx].forward(x[:, j, :].unsqueeze(1))
                output_accum += conv_result.squeeze(1)  # Squeeze
            output[:, i, :] = output_accum  # A to output channel

        return output

class KAN_Convolution_1D(torch.nn.Module):
    def __init__(
            self,
            kernel_size: int = 2,
            stride: int = 1,
            padding: int = 0,
            dilation: int = 1,
            grid_size: int = 50,
            spline_order: int = 3,
            scale_noise: float = 0.1,
            scale_base: float = 1.0,
            scale_spline: float = 1.0,
            base_activation=torch.nn.SiLU,
            grid_eps: float = 0.02,
            grid_range: tuple = [-1, 1]
        ):
        super(KAN_Convolution_1D, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.conv = KANLinear(
            in_features = kernel_size,
            out_features = 1,
            grid_size=grid_size,
            spline_order=spline_order,
            scale_noise=scale_noise,
            scale_base=scale_base,
            scale_spline=scale_spline,
            base_activation=base_activation,
            grid_eps=grid_eps,
            grid_range=grid_range
        )

    def forward(self, x: torch.Tensor):
        self.device = x.device
        return kan_conv1d(x, self.conv, self.kernel_size,self.stride, self.dilation, self.padding, self.device)

Example with changed mapper

In [ ]:
# class SeqNN(nn.Module):
#     """
#     SeqNN neural network for binary classification with modified mapper using KAN_Convolutional_Layer_1D.
#     """
#     def __init__(self,
#                  seqsize,
#                  use_single_channel,
#                  use_reverse_channel,
#                  use_multisubstate_channel,
#                  block_sizes=[256, 256, 128, 128, 64, 64, 32, 32],
#                  ks=5,
#                  resize_factor=4,
#                  activation=nn.SiLU,
#                  filter_per_group=2,
#                  se_reduction=4,
#                  final_ch=1,
#                  bn_momentum=0.1):
#         super().__init__()

#         self.block_sizes = block_sizes
#         self.resize_factor = resize_factor
#         self.se_reduction = se_reduction
#         self.seqsize = seqsize
#         self.use_single_channel = use_single_channel
#         self.use_reverse_channel = use_reverse_channel
#         self.use_multisubstate_channel = use_multisubstate_channel
#         self.final_ch = final_ch
#         self.bn_momentum = bn_momentum

#         seqextblocks = OrderedDict()
#         in_channels_first_block = 4  # 4 channels for A, T, C, G
#         if self.use_single_channel:
#             in_channels_first_block += 1
#         if self.use_reverse_channel:
#             in_channels_first_block += 1
#         if self.use_multisubstate_channel:
#             in_channels_first_block += 1

#         # First layer
#         block = nn.Sequential(
#             nn.Conv1d(
#                 in_channels=in_channels_first_block,
#                 out_channels=block_sizes[0],
#                 kernel_size=ks,
#                 padding='same',
#                 bias=False
#             ),
#             nn.BatchNorm1d(block_sizes[0], momentum=self.bn_momentum),
#             activation()
#         )
#         seqextblocks['blc0'] = block

#         # Building remaining blocks
#         for ind, (prev_sz, sz) in enumerate(zip(block_sizes[:-1], block_sizes[1:])):
#             block = nn.Sequential(
#                 nn.Conv1d(prev_sz, sz * self.resize_factor, kernel_size=1, padding='same', bias=False),
#                 nn.BatchNorm1d(sz * self.resize_factor, momentum=self.bn_momentum),
#                 activation(),

#                 nn.Conv1d(sz * self.resize_factor, sz * self.resize_factor, kernel_size=ks,
#                           groups=sz * self.resize_factor // filter_per_group, padding='same', bias=False),
#                 nn.BatchNorm1d(sz * self.resize_factor, momentum=self.bn_momentum),
#                 activation(),

#                 SELayer(prev_sz, sz * self.resize_factor, reduction=self.se_reduction),

#                 nn.Conv1d(sz * self.resize_factor, prev_sz, kernel_size=1, padding='same', bias=False),
#                 nn.BatchNorm1d(prev_sz, momentum=self.bn_momentum),
#                 activation(),
#             )
#             seqextblocks[f'inv_res_blc{ind}'] = block

#             resize_block = nn.Sequential(
#                 nn.Conv1d(2 * prev_sz, sz, kernel_size=ks, padding='same', bias=False),
#                 nn.BatchNorm1d(sz, momentum=self.bn_momentum),
#                 activation()
#             )
#             seqextblocks[f'resize_blc{ind}'] = resize_block

#         self.seqextractor = nn.ModuleDict(seqextblocks)

#         self.mapper = nn.Sequential(
#             KAN_Convolutional_Layer_1D(
#                 in_channels=block_sizes[-1],
#                 out_channels=self.final_ch,
#                 kernel_size=1,
#                 padding=0,
#                 device="cuda"
#             ),
#             activation()
#         )

#         self.register_buffer('bins', torch.arange(start=0, end=self.final_ch, step=1, requires_grad=False))

#     def feature_extractor(self, x):
#         x = self.seqextractor['blc0'](x)
#         for i in range(len(self.block_sizes) - 1):
#             x = torch.cat([x, self.seqextractor[f'inv_res_blc{i}'](x)], dim=1)
#             x = self.seqextractor[f'resize_blc{i}'](x)
#         return x

#     def forward(self, x):
#         f = self.feature_extractor(x)
#         x = self.mapper(f)
#         x = F.adaptive_avg_pool1d(x, 1)
#         x = x.squeeze(2)
#         prob = torch.sigmoid(x).squeeze(1)

#         return prob

In [ ]:
device = 'cuda'

In [ ]:
model = SeqNN(
    seqsize=500,
    use_single_channel=False,
    use_reverse_channel=False,
    use_multisubstate_channel=False,
    final_ch=1
)

criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.005)


In [ ]:
model.to(device)

SeqNN(
  (seqextractor): ModuleDict(
    (blc0): Sequential(
      (0): Conv1d(4, 256, kernel_size=(5,), stride=(1,), padding=same, bias=False)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
    )
    (inv_res_blc0): Sequential(
      (0): Conv1d(256, 1024, kernel_size=(1,), stride=(1,), padding=same, bias=False)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
      (3): Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=same, groups=512, bias=False)
      (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): SiLU()
      (6): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): KANLinear(
            (base_activation): SiLU()
          )
          (1): KANLinear(
            (base_activation): SiLU()
          )
          (2): Sigmoid()
        )
      )
      (

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 4990177


In [ ]:
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for sequences, labels in loader:
        sequences, labels = sequences.to(device), labels.to(device)
        optimizer.zero_grad()

        probs = model(sequences)
        loss = criterion(probs, labels.float())

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * sequences.size(0)
        all_preds.extend(probs.detach().cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(loader.dataset)

    all_preds = [1 if p >= 0.5 else 0 for p in all_preds]
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return epoch_loss, accuracy, f1, auc, precision, recall, mcc


def evaluate(model, loader, criterion, device):
    model.eval()
    eval_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for sequences, labels in loader:
            sequences, labels = sequences.to(device), labels.to(device)


            probs = model(sequences)
            loss = criterion(probs, labels.float())
            eval_loss += loss.item() * sequences.size(0)

            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = eval_loss / len(loader.dataset)

    all_preds = [1 if p >= 0.5 else 0 for p in all_preds]
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return epoch_loss, accuracy, f1, auc, precision, recall, mcc

#EMP

H3

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")


Epoch 1/10
Train - Loss: 0.4987, Acc: 0.8272, F1: 0.8297, AUC: 0.8274, Precision: 0.8401, Recall: 0.8195, MCC: 0.6545
Val   - Loss: 0.5041, Acc: 0.8176, F1: 0.8037, AUC: 0.8183, Precision: 0.8789, Recall: 0.7404, MCC: 0.6439
Epoch 2/10
Train - Loss: 0.4847, Acc: 0.8364, F1: 0.8391, AUC: 0.8365, Precision: 0.8476, Recall: 0.8309, MCC: 0.6728
Val   - Loss: 0.5011, Acc: 0.8390, F1: 0.8375, AUC: 0.8392, Precision: 0.8530, Recall: 0.8225, MCC: 0.6785
Epoch 3/10
Train - Loss: 0.4686, Acc: 0.8485, F1: 0.8516, AUC: 0.8485, Precision: 0.8568, Recall: 0.8465, MCC: 0.6969
Val   - Loss: 0.4846, Acc: 0.8564, F1: 0.8608, AUC: 0.8562, Precision: 0.8418, Recall: 0.8808, MCC: 0.7134
Epoch 4/10
Train - Loss: 0.4654, Acc: 0.8529, F1: 0.8557, AUC: 0.8530, Precision: 0.8624, Recall: 0.8491, MCC: 0.7058
Val   - Loss: 0.4885, Acc: 0.8424, F1: 0.8333, AUC: 0.8429, Precision: 0.8926, Recall: 0.7815, MCC: 0.6905
Epoch 5/10
Train - Loss: 0.4576, Acc: 0.8567, F1: 0.8584, AUC: 0.8571, Precision: 0.8718, Recall: 0.

H3K4me1

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")


Epoch 1/10
Train - Loss: 0.6443, Acc: 0.6389, F1: 0.6897, AUC: 0.6292, Precision: 0.6499, Recall: 0.7348, MCC: 0.2646
Val   - Loss: 0.6268, Acc: 0.6686, F1: 0.7103, AUC: 0.6609, Precision: 0.6810, Recall: 0.7422, MCC: 0.3264
Epoch 2/10
Train - Loss: 0.6288, Acc: 0.6641, F1: 0.6971, AUC: 0.6596, Precision: 0.6868, Recall: 0.7078, MCC: 0.3204
Val   - Loss: 0.6269, Acc: 0.6736, F1: 0.7158, AUC: 0.6655, Precision: 0.6838, Recall: 0.7509, MCC: 0.3365
Epoch 3/10
Train - Loss: 0.6240, Acc: 0.6687, F1: 0.7028, AUC: 0.6638, Precision: 0.6890, Recall: 0.7171, MCC: 0.3292
Val   - Loss: 0.6282, Acc: 0.6689, F1: 0.7293, AUC: 0.6536, Precision: 0.6600, Recall: 0.8149, MCC: 0.3267
Epoch 4/10
Train - Loss: 0.6190, Acc: 0.6738, F1: 0.7038, AUC: 0.6702, Precision: 0.6982, Recall: 0.7095, MCC: 0.3411
Val   - Loss: 0.6321, Acc: 0.6578, F1: 0.7309, AUC: 0.6378, Precision: 0.6417, Recall: 0.8489, MCC: 0.3070
Epoch 5/10
Train - Loss: 0.6141, Acc: 0.6809, F1: 0.7071, AUC: 0.6784, Precision: 0.7090, Recall: 0.

H3K4me2

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6365, Acc: 0.6420, F1: 0.7344, AUC: 0.5980, Precision: 0.6554, Recall: 0.8351, MCC: 0.2240
Val   - Loss: 0.6344, Acc: 0.6562, F1: 0.7564, AUC: 0.6038, Precision: 0.6459, Recall: 0.9125, MCC: 0.2701
Epoch 2/10
Train - Loss: 0.6305, Acc: 0.6540, F1: 0.7364, AUC: 0.6173, Precision: 0.6714, Recall: 0.8152, MCC: 0.2565
Val   - Loss: 0.6313, Acc: 0.6615, F1: 0.7368, AUC: 0.6311, Precision: 0.6757, Recall: 0.8100, MCC: 0.2822
Epoch 3/10
Train - Loss: 0.6287, Acc: 0.6541, F1: 0.7409, AUC: 0.6130, Precision: 0.6663, Recall: 0.8342, MCC: 0.2539
Val   - Loss: 0.6301, Acc: 0.6572, F1: 0.7243, AUC: 0.6342, Precision: 0.6838, Recall: 0.7699, MCC: 0.2788
Epoch 4/10
Train - Loss: 0.6246, Acc: 0.6570, F1: 0.7383, AUC: 0.6208, Precision: 0.6740, Recall: 0.8161, MCC: 0.2637
Val   - Loss: 0.6321, Acc: 0.6540, F1: 0.7106, AUC: 0.6391, Precision: 0.6955, Recall: 0.7265, MCC: 0.2812
Epoch 5/10
Train - Loss: 0.6214, Acc: 0.6586, F1: 0.7395, AUC: 0.6224, Precision: 0.6751, Recall: 0.

H3K4me3

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6702, Acc: 0.5921, F1: 0.6591, AUC: 0.5816, Precision: 0.5946, Recall: 0.7394, MCC: 0.1722
Val   - Loss: 0.6669, Acc: 0.6114, F1: 0.6691, AUC: 0.6032, Precision: 0.6094, Recall: 0.7419, MCC: 0.2153
Epoch 2/10
Train - Loss: 0.6588, Acc: 0.6162, F1: 0.6598, AUC: 0.6103, Precision: 0.6257, Recall: 0.6980, MCC: 0.2243
Val   - Loss: 0.6578, Acc: 0.6168, F1: 0.6320, AUC: 0.6166, Precision: 0.6431, Recall: 0.6213, MCC: 0.2328
Epoch 3/10
Train - Loss: 0.6505, Acc: 0.6294, F1: 0.6656, AUC: 0.6249, Precision: 0.6415, Recall: 0.6916, MCC: 0.2521
Val   - Loss: 0.6632, Acc: 0.6003, F1: 0.5492, AUC: 0.6091, Precision: 0.6819, Recall: 0.4597, MCC: 0.2273
Epoch 4/10
Train - Loss: 0.6424, Acc: 0.6395, F1: 0.6666, AUC: 0.6369, Precision: 0.6576, Recall: 0.6759, MCC: 0.2744
Val   - Loss: 0.6525, Acc: 0.6318, F1: 0.6658, AUC: 0.6280, Precision: 0.6410, Recall: 0.6927, MCC: 0.2582
Epoch 5/10
Train - Loss: 0.6359, Acc: 0.6473, F1: 0.6699, AUC: 0.6456, Precision: 0.6688, Recall: 0.

H3K9ac

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.5572, Acc: 0.7357, F1: 0.7583, AUC: 0.7345, Precision: 0.7718, Recall: 0.7452, MCC: 0.4673
Val   - Loss: 0.5517, Acc: 0.7460, F1: 0.7667, AUC: 0.7448, Precision: 0.7780, Recall: 0.7557, MCC: 0.4882
Epoch 2/10
Train - Loss: 0.5502, Acc: 0.7452, F1: 0.7666, AUC: 0.7443, Precision: 0.7815, Recall: 0.7523, MCC: 0.4867
Val   - Loss: 0.5652, Acc: 0.7283, F1: 0.7199, AUC: 0.7396, Precision: 0.8362, Recall: 0.6319, MCC: 0.4832
Epoch 3/10
Train - Loss: 0.5469, Acc: 0.7474, F1: 0.7662, AUC: 0.7478, Precision: 0.7896, Recall: 0.7442, MCC: 0.4930
Val   - Loss: 0.5547, Acc: 0.7438, F1: 0.7612, AUC: 0.7443, Precision: 0.7844, Recall: 0.7394, MCC: 0.4863
Epoch 4/10
Train - Loss: 0.5413, Acc: 0.7515, F1: 0.7695, AUC: 0.7522, Precision: 0.7949, Recall: 0.7456, MCC: 0.5017
Val   - Loss: 0.5477, Acc: 0.7474, F1: 0.7566, AUC: 0.7517, Precision: 0.8087, Recall: 0.7107, MCC: 0.5008
Epoch 5/10
Train - Loss: 0.5347, Acc: 0.7568, F1: 0.7731, AUC: 0.7584, Precision: 0.8037, Recall: 0.

H3K14ac

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6281, Acc: 0.6510, F1: 0.7096, AUC: 0.6355, Precision: 0.6701, Recall: 0.7540, MCC: 0.2793
Val   - Loss: 0.6298, Acc: 0.6545, F1: 0.7562, AUC: 0.6061, Precision: 0.6352, Recall: 0.9341, MCC: 0.2890
Epoch 2/10
Train - Loss: 0.6089, Acc: 0.6815, F1: 0.7255, AUC: 0.6720, Precision: 0.7076, Recall: 0.7443, MCC: 0.3474
Val   - Loss: 0.6052, Acc: 0.6989, F1: 0.7508, AUC: 0.6831, Precision: 0.7148, Recall: 0.7906, MCC: 0.3761
Epoch 3/10
Train - Loss: 0.6015, Acc: 0.6912, F1: 0.7319, AUC: 0.6831, Precision: 0.7190, Recall: 0.7452, MCC: 0.3686
Val   - Loss: 0.6020, Acc: 0.6862, F1: 0.7290, AUC: 0.6777, Precision: 0.7224, Recall: 0.7358, MCC: 0.3566
Epoch 4/10
Train - Loss: 0.5969, Acc: 0.6963, F1: 0.7339, AUC: 0.6897, Precision: 0.7274, Recall: 0.7404, MCC: 0.3805
Val   - Loss: 0.6000, Acc: 0.6983, F1: 0.7358, AUC: 0.6925, Precision: 0.7395, Recall: 0.7321, MCC: 0.3844
Epoch 5/10
Train - Loss: 0.5899, Acc: 0.7008, F1: 0.7370, AUC: 0.6947, Precision: 0.7327, Recall: 0.

H3K36me3

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6094, Acc: 0.6872, F1: 0.7145, AUC: 0.6840, Precision: 0.7055, Recall: 0.7237, MCC: 0.3689
Val   - Loss: 0.6149, Acc: 0.6694, F1: 0.6809, AUC: 0.6741, Precision: 0.7361, Recall: 0.6334, MCC: 0.3462
Epoch 2/10
Train - Loss: 0.5891, Acc: 0.7136, F1: 0.7340, AUC: 0.7121, Precision: 0.7372, Recall: 0.7309, MCC: 0.4238
Val   - Loss: 0.5879, Acc: 0.7116, F1: 0.7337, AUC: 0.7113, Precision: 0.7549, Recall: 0.7137, MCC: 0.4205
Epoch 3/10
Train - Loss: 0.5832, Acc: 0.7201, F1: 0.7403, AUC: 0.7186, Precision: 0.7430, Recall: 0.7376, MCC: 0.4369
Val   - Loss: 0.6025, Acc: 0.6697, F1: 0.6368, AUC: 0.6889, Precision: 0.8211, Recall: 0.5201, MCC: 0.3928
Epoch 4/10
Train - Loss: 0.5776, Acc: 0.7266, F1: 0.7461, AUC: 0.7252, Precision: 0.7494, Recall: 0.7429, MCC: 0.4500
Val   - Loss: 0.5774, Acc: 0.7308, F1: 0.7599, AUC: 0.7264, Precision: 0.7547, Recall: 0.7652, MCC: 0.4536
Epoch 5/10
Train - Loss: 0.5724, Acc: 0.7311, F1: 0.7499, AUC: 0.7299, Precision: 0.7544, Recall: 0.

H3K79me3

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.5702, Acc: 0.7437, F1: 0.7649, AUC: 0.7409, Precision: 0.7456, Recall: 0.7851, MCC: 0.4844
Val   - Loss: 0.5521, Acc: 0.7677, F1: 0.7990, AUC: 0.7612, Precision: 0.7367, Recall: 0.8729, MCC: 0.5391
Epoch 2/10
Train - Loss: 0.5408, Acc: 0.7826, F1: 0.7970, AUC: 0.7812, Precision: 0.7903, Recall: 0.8038, MCC: 0.5631
Val   - Loss: 0.5306, Acc: 0.7902, F1: 0.7995, AUC: 0.7902, Precision: 0.8089, Recall: 0.7903, MCC: 0.5798
Epoch 3/10
Train - Loss: 0.5364, Acc: 0.7812, F1: 0.7934, AUC: 0.7805, Precision: 0.7955, Recall: 0.7913, MCC: 0.5608
Val   - Loss: 0.5371, Acc: 0.7926, F1: 0.8175, AUC: 0.7874, Precision: 0.7651, Recall: 0.8775, MCC: 0.5874
Epoch 4/10
Train - Loss: 0.5302, Acc: 0.7889, F1: 0.8023, AUC: 0.7878, Precision: 0.7981, Recall: 0.8066, MCC: 0.5760
Val   - Loss: 0.5334, Acc: 0.7764, F1: 0.7657, AUC: 0.7817, Precision: 0.8590, Recall: 0.6907, MCC: 0.5687
Epoch 5/10
Train - Loss: 0.5244, Acc: 0.7954, F1: 0.8072, AUC: 0.7946, Precision: 0.8077, Recall: 0.

H4

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.5467, Acc: 0.8123, F1: 0.7930, AUC: 0.8119, Precision: 0.7784, Recall: 0.8081, MCC: 0.6218
Val   - Loss: 0.5131, Acc: 0.8412, F1: 0.8034, AUC: 0.8301, Precision: 0.8893, Recall: 0.7326, MCC: 0.6812
Epoch 2/10
Train - Loss: 0.5126, Acc: 0.8514, F1: 0.8305, AUC: 0.8482, Precision: 0.8430, Recall: 0.8185, MCC: 0.6986
Val   - Loss: 0.4821, Acc: 0.8775, F1: 0.8603, AUC: 0.8748, Precision: 0.8691, Recall: 0.8516, MCC: 0.7513
Epoch 3/10
Train - Loss: 0.5040, Acc: 0.8593, F1: 0.8393, AUC: 0.8560, Precision: 0.8534, Recall: 0.8256, MCC: 0.7146
Val   - Loss: 0.4827, Acc: 0.8754, F1: 0.8613, AUC: 0.8752, Precision: 0.8496, Recall: 0.8733, MCC: 0.7485
Epoch 4/10
Train - Loss: 0.4928, Acc: 0.8650, F1: 0.8468, AUC: 0.8624, Precision: 0.8548, Recall: 0.8389, MCC: 0.7262
Val   - Loss: 0.4882, Acc: 0.8576, F1: 0.8228, AUC: 0.8462, Precision: 0.9165, Recall: 0.7465, MCC: 0.7163
Epoch 5/10
Train - Loss: 0.4831, Acc: 0.8705, F1: 0.8510, AUC: 0.8666, Precision: 0.8719, Recall: 0.

H4ac

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6412, Acc: 0.6436, F1: 0.6951, AUC: 0.6339, Precision: 0.6476, Recall: 0.7501, MCC: 0.2760
Val   - Loss: 0.6310, Acc: 0.6651, F1: 0.7066, AUC: 0.6587, Precision: 0.6620, Recall: 0.7576, MCC: 0.3246
Epoch 2/10
Train - Loss: 0.6226, Acc: 0.6684, F1: 0.7010, AUC: 0.6639, Precision: 0.6850, Recall: 0.7178, MCC: 0.3298
Val   - Loss: 0.6258, Acc: 0.6589, F1: 0.6521, AUC: 0.6630, Precision: 0.7134, Recall: 0.6006, MCC: 0.3270
Epoch 3/10
Train - Loss: 0.6148, Acc: 0.6792, F1: 0.7076, AUC: 0.6758, Precision: 0.6988, Recall: 0.7166, MCC: 0.3526
Val   - Loss: 0.6283, Acc: 0.6633, F1: 0.6294, AUC: 0.6720, Precision: 0.7599, Recall: 0.5372, MCC: 0.3544
Epoch 4/10
Train - Loss: 0.6074, Acc: 0.6877, F1: 0.7128, AUC: 0.6852, Precision: 0.7102, Recall: 0.7153, MCC: 0.3707
Val   - Loss: 0.6146, Acc: 0.6765, F1: 0.7173, AUC: 0.6700, Precision: 0.6707, Recall: 0.7708, MCC: 0.3482
Epoch 5/10
Train - Loss: 0.6017, Acc: 0.6919, F1: 0.7144, AUC: 0.6901, Precision: 0.7173, Recall: 0.

# TFP Human


0

In [ ]:
for epoch in range(num_epochs):
    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.5802, Acc: 0.7434, F1: 0.7438, AUC: 0.7434, Precision: 0.7419, Recall: 0.7457, MCC: 0.4868
Val   - Loss: 0.5288, Acc: 0.8070, F1: 0.8083, AUC: 0.8070, Precision: 0.8028, Recall: 0.8140, MCC: 0.6141
Epoch 2/10
Train - Loss: 0.5600, Acc: 0.7652, F1: 0.7627, AUC: 0.7652, Precision: 0.7701, Recall: 0.7554, MCC: 0.5305
Val   - Loss: 0.5259, Acc: 0.8090, F1: 0.8265, AUC: 0.8090, Precision: 0.7571, Recall: 0.9100, MCC: 0.6310
Epoch 3/10
Train - Loss: 0.5525, Acc: 0.7717, F1: 0.7709, AUC: 0.7717, Precision: 0.7728, Recall: 0.7691, MCC: 0.5435
Val   - Loss: 0.5215, Acc: 0.8140, F1: 0.8309, AUC: 0.8140, Precision: 0.7617, Recall: 0.9140, MCC: 0.6409
Epoch 4/10
Train - Loss: 0.5475, Acc: 0.7778, F1: 0.7767, AUC: 0.7778, Precision: 0.7797, Recall: 0.7737, MCC: 0.5556
Val   - Loss: 0.5100, Acc: 0.8300, F1: 0.8353, AUC: 0.8300, Precision: 0.8102, Recall: 0.8620, MCC: 0.6614
Epoch 5/10
Train - Loss: 0.5394, Acc: 0.7851, F1: 0.7840, AUC: 0.7851, Precision: 0.7872, Recall: 0.

1

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

2

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

3

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

4

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

# TFP Mouse

0

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6407, Acc: 0.6445, F1: 0.6926, AUC: 0.6359, Precision: 0.6514, Recall: 0.7393, MCC: 0.2784
Val   - Loss: 0.6349, Acc: 0.6584, F1: 0.7038, AUC: 0.6512, Precision: 0.6534, Recall: 0.7625, MCC: 0.3110
Epoch 2/10
Train - Loss: 0.6232, Acc: 0.6714, F1: 0.7031, AUC: 0.6671, Precision: 0.6884, Recall: 0.7185, MCC: 0.3359
Val   - Loss: 0.6229, Acc: 0.6630, F1: 0.7083, AUC: 0.6558, Precision: 0.6568, Recall: 0.7686, MCC: 0.3207
Epoch 3/10
Train - Loss: 0.6158, Acc: 0.6815, F1: 0.7134, AUC: 0.6769, Precision: 0.6958, Recall: 0.7319, MCC: 0.3561
Val   - Loss: 0.6179, Acc: 0.6745, F1: 0.6805, AUC: 0.6761, Precision: 0.7125, Recall: 0.6512, MCC: 0.3516
Epoch 4/10
Train - Loss: 0.6089, Acc: 0.6864, F1: 0.7167, AUC: 0.6822, Precision: 0.7015, Recall: 0.7326, MCC: 0.3663
Val   - Loss: 0.6348, Acc: 0.6595, F1: 0.7316, AUC: 0.6449, Precision: 0.6303, Recall: 0.8716, MCC: 0.3281
Epoch 5/10
Train - Loss: 0.6029, Acc: 0.6953, F1: 0.7212, AUC: 0.6923, Precision: 0.7148, Recall: 0.

1

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.4525, Acc: 0.8492, F1: 0.8415, AUC: 0.8492, Precision: 0.8864, Recall: 0.8008, MCC: 0.7017
Val   - Loss: 0.4157, Acc: 0.8772, F1: 0.8774, AUC: 0.8772, Precision: 0.8763, Recall: 0.8784, MCC: 0.7545
Epoch 2/10
Train - Loss: 0.4216, Acc: 0.8763, F1: 0.8703, AUC: 0.8763, Precision: 0.9145, Recall: 0.8301, MCC: 0.7559
Val   - Loss: 0.4174, Acc: 0.8824, F1: 0.8812, AUC: 0.8824, Precision: 0.8904, Recall: 0.8722, MCC: 0.7650
Epoch 3/10
Train - Loss: 0.4114, Acc: 0.8875, F1: 0.8835, AUC: 0.8875, Precision: 0.9155, Recall: 0.8536, MCC: 0.7767
Val   - Loss: 0.4024, Acc: 0.8930, F1: 0.8868, AUC: 0.8929, Precision: 0.9405, Recall: 0.8390, MCC: 0.7905
Epoch 4/10
Train - Loss: 0.4048, Acc: 0.8943, F1: 0.8908, AUC: 0.8943, Precision: 0.9206, Recall: 0.8630, MCC: 0.7902
Val   - Loss: 0.4046, Acc: 0.8946, F1: 0.8898, AUC: 0.8946, Precision: 0.9324, Recall: 0.8508, MCC: 0.7922
Epoch 5/10
Train - Loss: 0.3985, Acc: 0.8970, F1: 0.8935, AUC: 0.8970, Precision: 0.9240, Recall: 0.

2

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.5783, Acc: 0.7630, F1: 0.7385, AUC: 0.7629, Precision: 0.8227, Recall: 0.6700, MCC: 0.5352
Val   - Loss: 0.5489, Acc: 0.7561, F1: 0.7546, AUC: 0.7561, Precision: 0.7593, Recall: 0.7500, MCC: 0.5122
Epoch 2/10
Train - Loss: 0.4778, Acc: 0.8637, F1: 0.8591, AUC: 0.8637, Precision: 0.8889, Recall: 0.8312, MCC: 0.7290
Val   - Loss: 0.4213, Acc: 0.8994, F1: 0.8881, AUC: 0.8994, Precision: 1.0000, Recall: 0.7988, MCC: 0.8155
Epoch 3/10
Train - Loss: 0.4411, Acc: 0.8656, F1: 0.8536, AUC: 0.8656, Precision: 0.9370, Recall: 0.7838, MCC: 0.7412
Val   - Loss: 0.4055, Acc: 0.9116, F1: 0.9068, AUC: 0.9116, Precision: 0.9592, Recall: 0.8598, MCC: 0.8276
Epoch 4/10
Train - Loss: 0.4168, Acc: 0.8855, F1: 0.8785, AUC: 0.8855, Precision: 0.9345, Recall: 0.8289, MCC: 0.7759
Val   - Loss: 0.3980, Acc: 0.9055, F1: 0.9016, AUC: 0.9055, Precision: 0.9404, Recall: 0.8659, MCC: 0.8135
Epoch 5/10
Train - Loss: 0.4115, Acc: 0.8882, F1: 0.8794, AUC: 0.8881, Precision: 0.9536, Recall: 0.

3

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.5741, Acc: 0.7397, F1: 0.6990, AUC: 0.7396, Precision: 0.8276, Recall: 0.6050, MCC: 0.4976
Val   - Loss: 0.6579, Acc: 0.6799, F1: 0.7506, AUC: 0.6799, Precision: 0.6148, Recall: 0.9634, MCC: 0.4368
Epoch 2/10
Train - Loss: 0.4805, Acc: 0.8538, F1: 0.8478, AUC: 0.8538, Precision: 0.8833, Recall: 0.8151, MCC: 0.7097
Val   - Loss: 0.4260, Acc: 0.8963, F1: 0.8910, AUC: 0.8963, Precision: 0.9392, Recall: 0.8476, MCC: 0.7965
Epoch 3/10
Train - Loss: 0.4427, Acc: 0.8748, F1: 0.8671, AUC: 0.8748, Precision: 0.9232, Recall: 0.8174, MCC: 0.7546
Val   - Loss: 0.3945, Acc: 0.9085, F1: 0.9026, AUC: 0.9085, Precision: 0.9653, Recall: 0.8476, MCC: 0.8232
Epoch 4/10
Train - Loss: 0.4162, Acc: 0.8889, F1: 0.8815, AUC: 0.8889, Precision: 0.9442, Recall: 0.8266, MCC: 0.7839
Val   - Loss: 0.3937, Acc: 0.9207, F1: 0.9172, AUC: 0.9207, Precision: 0.9600, Recall: 0.8780, MCC: 0.8445
Epoch 5/10
Train - Loss: 0.4169, Acc: 0.8851, F1: 0.8763, AUC: 0.8851, Precision: 0.9484, Recall: 0.

4

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6332, Acc: 0.6508, F1: 0.6488, AUC: 0.6507, Precision: 0.6493, Recall: 0.6483, MCC: 0.3015
Val   - Loss: 0.6204, Acc: 0.6824, F1: 0.6495, AUC: 0.6824, Precision: 0.7242, Recall: 0.5887, MCC: 0.3713
Epoch 2/10
Train - Loss: 0.5984, Acc: 0.7071, F1: 0.6909, AUC: 0.7069, Precision: 0.7274, Recall: 0.6580, MCC: 0.4158
Val   - Loss: 0.6082, Acc: 0.6973, F1: 0.6919, AUC: 0.6973, Precision: 0.7041, Recall: 0.6801, MCC: 0.3948
Epoch 3/10
Train - Loss: 0.5817, Acc: 0.7301, F1: 0.7206, AUC: 0.7299, Precision: 0.7430, Recall: 0.6996, MCC: 0.4608
Val   - Loss: 0.5933, Acc: 0.7329, F1: 0.7117, AUC: 0.7328, Precision: 0.7724, Recall: 0.6599, MCC: 0.4707
Epoch 4/10
Train - Loss: 0.5742, Acc: 0.7299, F1: 0.7164, AUC: 0.7297, Precision: 0.7500, Recall: 0.6857, MCC: 0.4612
Val   - Loss: 0.6229, Acc: 0.7047, F1: 0.7385, AUC: 0.7048, Precision: 0.6624, Recall: 0.8342, MCC: 0.4240
Epoch 5/10
Train - Loss: 0.5670, Acc: 0.7432, F1: 0.7306, AUC: 0.7430, Precision: 0.7644, Recall: 0.

# Promoter Detection

prom_300_all

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.4702, Acc: 0.8627, F1: 0.8619, AUC: 0.8627, Precision: 0.8669, Recall: 0.8570, MCC: 0.7255
Val   - Loss: 0.4541, Acc: 0.8792, F1: 0.8778, AUC: 0.8793, Precision: 0.8945, Recall: 0.8617, MCC: 0.7590
Epoch 2/10
Train - Loss: 0.4482, Acc: 0.8851, F1: 0.8861, AUC: 0.8851, Precision: 0.8777, Recall: 0.8948, MCC: 0.7703
Val   - Loss: 0.4557, Acc: 0.8796, F1: 0.8882, AUC: 0.8791, Precision: 0.8335, Recall: 0.9507, MCC: 0.7666
Epoch 3/10
Train - Loss: 0.4402, Acc: 0.8917, F1: 0.8931, AUC: 0.8917, Precision: 0.8817, Recall: 0.9048, MCC: 0.7837
Val   - Loss: 0.4417, Acc: 0.8867, F1: 0.8877, AUC: 0.8866, Precision: 0.8852, Recall: 0.8903, MCC: 0.7733
Epoch 4/10
Train - Loss: 0.4360, Acc: 0.8939, F1: 0.8952, AUC: 0.8939, Precision: 0.8836, Recall: 0.9071, MCC: 0.7880
Val   - Loss: 0.4541, Acc: 0.8858, F1: 0.8869, AUC: 0.8858, Precision: 0.8845, Recall: 0.8893, MCC: 0.7716
Epoch 5/10
Train - Loss: 0.4309, Acc: 0.8964, F1: 0.8979, AUC: 0.8964, Precision: 0.8845, Recall: 0.

prom_300_notata

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.3916, Acc: 0.9101, F1: 0.9078, AUC: 0.9101, Precision: 0.9316, Recall: 0.8852, MCC: 0.8212
Val   - Loss: 0.3741, Acc: 0.9148, F1: 0.9104, AUC: 0.9155, Precision: 0.9725, Recall: 0.8557, MCC: 0.8360
Epoch 2/10
Train - Loss: 0.3656, Acc: 0.9304, F1: 0.9286, AUC: 0.9304, Precision: 0.9525, Recall: 0.9059, MCC: 0.8618
Val   - Loss: 0.3578, Acc: 0.9324, F1: 0.9316, AUC: 0.9326, Precision: 0.9521, Recall: 0.9120, MCC: 0.8655
Epoch 3/10
Train - Loss: 0.3562, Acc: 0.9383, F1: 0.9371, AUC: 0.9383, Precision: 0.9554, Recall: 0.9194, MCC: 0.8771
Val   - Loss: 0.3621, Acc: 0.9324, F1: 0.9338, AUC: 0.9322, Precision: 0.9244, Recall: 0.9433, MCC: 0.8648
Epoch 4/10
Train - Loss: 0.3535, Acc: 0.9388, F1: 0.9376, AUC: 0.9388, Precision: 0.9573, Recall: 0.9186, MCC: 0.8784
Val   - Loss: 0.3571, Acc: 0.9357, F1: 0.9361, AUC: 0.9358, Precision: 0.9405, Recall: 0.9318, MCC: 0.8715
Epoch 5/10
Train - Loss: 0.3490, Acc: 0.9431, F1: 0.9422, AUC: 0.9431, Precision: 0.9573, Recall: 0.

prom_300_tata

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6572, Acc: 0.6305, F1: 0.6389, AUC: 0.6306, Precision: 0.6228, Recall: 0.6559, MCC: 0.2615
Val   - Loss: 0.6687, Acc: 0.6232, F1: 0.6916, AUC: 0.6298, Precision: 0.5743, Recall: 0.8691, MCC: 0.2943
Epoch 2/10
Train - Loss: 0.6198, Acc: 0.6905, F1: 0.6887, AUC: 0.6904, Precision: 0.6904, Recall: 0.6870, MCC: 0.3809
Val   - Loss: 0.6086, Acc: 0.6900, F1: 0.6690, AUC: 0.6888, Precision: 0.6957, Recall: 0.6443, MCC: 0.3794
Epoch 3/10
Train - Loss: 0.5980, Acc: 0.7164, F1: 0.7094, AUC: 0.7163, Precision: 0.7247, Recall: 0.6948, MCC: 0.4330
Val   - Loss: 0.5927, Acc: 0.7080, F1: 0.6384, AUC: 0.7032, Precision: 0.8020, Recall: 0.5302, MCC: 0.4349
Epoch 4/10
Train - Loss: 0.5796, Acc: 0.7331, F1: 0.7176, AUC: 0.7329, Precision: 0.7590, Recall: 0.6804, MCC: 0.4685
Val   - Loss: 0.5962, Acc: 0.6933, F1: 0.6357, AUC: 0.6895, Precision: 0.7523, Recall: 0.5503, MCC: 0.3956
Epoch 5/10
Train - Loss: 0.5663, Acc: 0.7500, F1: 0.7371, AUC: 0.7498, Precision: 0.7743, Recall: 0.

prom_core_all

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.5412, Acc: 0.7757, F1: 0.7674, AUC: 0.7756, Precision: 0.7958, Recall: 0.7410, MCC: 0.5526
Val   - Loss: 0.5126, Acc: 0.8064, F1: 0.7925, AUC: 0.8066, Precision: 0.8567, Recall: 0.7372, MCC: 0.6191
Epoch 2/10
Train - Loss: 0.5086, Acc: 0.8117, F1: 0.8059, AUC: 0.8117, Precision: 0.8305, Recall: 0.7828, MCC: 0.6245
Val   - Loss: 0.5120, Acc: 0.8081, F1: 0.7874, AUC: 0.8084, Precision: 0.8855, Recall: 0.7089, MCC: 0.6291
Epoch 3/10
Train - Loss: 0.5006, Acc: 0.8191, F1: 0.8132, AUC: 0.8191, Precision: 0.8398, Recall: 0.7882, MCC: 0.6395
Val   - Loss: 0.4993, Acc: 0.8191, F1: 0.8090, AUC: 0.8192, Precision: 0.8594, Recall: 0.7642, MCC: 0.6423
Epoch 4/10
Train - Loss: 0.4958, Acc: 0.8262, F1: 0.8214, AUC: 0.8262, Precision: 0.8439, Recall: 0.8000, MCC: 0.6533
Val   - Loss: 0.4947, Acc: 0.8265, F1: 0.8308, AUC: 0.8265, Precision: 0.8128, Recall: 0.8497, MCC: 0.6537
Epoch 5/10
Train - Loss: 0.4914, Acc: 0.8283, F1: 0.8228, AUC: 0.8282, Precision: 0.8489, Recall: 0.

prom_core_notata

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.5247, Acc: 0.7974, F1: 0.7926, AUC: 0.7973, Precision: 0.8095, Recall: 0.7765, MCC: 0.5952
Val   - Loss: 0.5094, Acc: 0.8146, F1: 0.8145, AUC: 0.8146, Precision: 0.8164, Recall: 0.8127, MCC: 0.6292
Epoch 2/10
Train - Loss: 0.5057, Acc: 0.8202, F1: 0.8171, AUC: 0.8202, Precision: 0.8293, Recall: 0.8053, MCC: 0.6407
Val   - Loss: 0.4936, Acc: 0.8304, F1: 0.8217, AUC: 0.8305, Precision: 0.8681, Recall: 0.7800, MCC: 0.6643
Epoch 3/10
Train - Loss: 0.4998, Acc: 0.8259, F1: 0.8223, AUC: 0.8258, Precision: 0.8372, Recall: 0.8079, MCC: 0.6521
Val   - Loss: 0.4940, Acc: 0.8249, F1: 0.8158, AUC: 0.8251, Precision: 0.8628, Recall: 0.7736, MCC: 0.6535
Epoch 4/10
Train - Loss: 0.4961, Acc: 0.8295, F1: 0.8275, AUC: 0.8295, Precision: 0.8348, Recall: 0.8203, MCC: 0.6590
Val   - Loss: 0.4931, Acc: 0.8257, F1: 0.8187, AUC: 0.8258, Precision: 0.8550, Recall: 0.7853, MCC: 0.6537
Epoch 5/10
Train - Loss: 0.4915, Acc: 0.8334, F1: 0.8304, AUC: 0.8333, Precision: 0.8430, Recall: 0.

prom_core_tata

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6234, Acc: 0.6786, F1: 0.6726, AUC: 0.6786, Precision: 0.6846, Recall: 0.6611, MCC: 0.3574
Val   - Loss: 0.5999, Acc: 0.7259, F1: 0.7447, AUC: 0.7254, Precision: 0.7020, Recall: 0.7929, MCC: 0.4552
Epoch 2/10
Train - Loss: 0.5598, Acc: 0.7661, F1: 0.7603, AUC: 0.7661, Precision: 0.7787, Recall: 0.7428, MCC: 0.5328
Val   - Loss: 0.4741, Acc: 0.8352, F1: 0.8279, AUC: 0.8356, Precision: 0.8741, Recall: 0.7864, MCC: 0.6742
Epoch 3/10
Train - Loss: 0.4528, Acc: 0.8479, F1: 0.8318, AUC: 0.8478, Precision: 0.9290, Recall: 0.7530, MCC: 0.7085
Val   - Loss: 0.4438, Acc: 0.8499, F1: 0.8403, AUC: 0.8505, Precision: 0.9064, Recall: 0.7832, MCC: 0.7068
Epoch 4/10
Train - Loss: 0.4259, Acc: 0.8636, F1: 0.8494, AUC: 0.8635, Precision: 0.9463, Recall: 0.7705, MCC: 0.7400
Val   - Loss: 0.4290, Acc: 0.8450, F1: 0.8211, AUC: 0.8462, Precision: 0.9820, Recall: 0.7055, MCC: 0.7202
Epoch 5/10
Train - Loss: 0.4205, Acc: 0.8662, F1: 0.8514, AUC: 0.8661, Precision: 0.9558, Recall: 0.

#Splice

In [ ]:
for epoch in range(10):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{10}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

#virus

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

#GUE+

0 (GM12878)

In [ ]:
num_epochs = 10

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6865, Acc: 0.5533, F1: 0.5704, AUC: 0.5558, Precision: 0.5296, Recall: 0.6180, MCC: 0.1123
Val   - Loss: 0.6825, Acc: 0.5685, F1: 0.5479, AUC: 0.5686, Precision: 0.5272, Recall: 0.5703, MCC: 0.1368
Epoch 2/10
Train - Loss: 0.6825, Acc: 0.5721, F1: 0.5571, AUC: 0.5717, Precision: 0.5535, Recall: 0.5607, MCC: 0.1433
Val   - Loss: 0.6777, Acc: 0.5860, F1: 0.6027, AUC: 0.5936, Precision: 0.5381, Recall: 0.6848, MCC: 0.1892
Epoch 3/10
Train - Loss: 0.6750, Acc: 0.5936, F1: 0.5725, AUC: 0.5926, Precision: 0.5781, Recall: 0.5670, MCC: 0.1853
Val   - Loss: 0.6762, Acc: 0.5910, F1: 0.3784, AUC: 0.5665, Precision: 0.6241, Recall: 0.2715, MCC: 0.1659
Epoch 4/10
Train - Loss: 0.6603, Acc: 0.6238, F1: 0.5801, AUC: 0.6206, Precision: 0.6246, Recall: 0.5416, MCC: 0.2445
Val   - Loss: 0.6699, Acc: 0.6030, F1: 0.3797, AUC: 0.5771, Precision: 0.6694, Recall: 0.2650, MCC: 0.1993
Epoch 5/10
Train - Loss: 0.6464, Acc: 0.6443, F1: 0.5982, AUC: 0.6407, Precision: 0.6532, Recall: 0.

1 (HeLa-S3)

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6844, Acc: 0.5645, F1: 0.5221, AUC: 0.5617, Precision: 0.5487, Recall: 0.4980, MCC: 0.1243
Val   - Loss: 0.6833, Acc: 0.5665, F1: 0.4950, AUC: 0.5615, Precision: 0.5599, Recall: 0.4436, MCC: 0.1267
Epoch 2/10
Train - Loss: 0.6780, Acc: 0.5819, F1: 0.5350, AUC: 0.5786, Precision: 0.5707, Recall: 0.5035, MCC: 0.1589
Val   - Loss: 0.6790, Acc: 0.5710, F1: 0.5541, AUC: 0.5704, Precision: 0.5518, Recall: 0.5564, MCC: 0.1408
Epoch 3/10
Train - Loss: 0.6707, Acc: 0.5984, F1: 0.5374, AUC: 0.5937, Precision: 0.5974, Recall: 0.4884, MCC: 0.1919
Val   - Loss: 0.6760, Acc: 0.5945, F1: 0.4703, AUC: 0.5857, Precision: 0.6283, Recall: 0.3758, MCC: 0.1893
Epoch 4/10
Train - Loss: 0.6622, Acc: 0.6083, F1: 0.5202, AUC: 0.6013, Precision: 0.6270, Recall: 0.4444, MCC: 0.2139
Val   - Loss: 0.6654, Acc: 0.5955, F1: 0.5726, AUC: 0.5943, Precision: 0.5797, Recall: 0.5658, MCC: 0.1888
Epoch 5/10
Train - Loss: 0.6469, Acc: 0.6331, F1: 0.5299, AUC: 0.6246, Precision: 0.6830, Recall: 0.

2 (HUVEC)

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Epoch 1/10
Train - Loss: 0.6798, Acc: 0.5798, F1: 0.5555, AUC: 0.5785, Precision: 0.5556, Recall: 0.5554, MCC: 0.1570
Val   - Loss: 0.6753, Acc: 0.5858, F1: 0.5241, AUC: 0.5833, Precision: 0.5984, Recall: 0.4663, MCC: 0.1714
Epoch 2/10
Train - Loss: 0.6699, Acc: 0.6042, F1: 0.5299, AUC: 0.5973, Precision: 0.6042, Recall: 0.4719, MCC: 0.2014
Val   - Loss: 0.6735, Acc: 0.5883, F1: 0.4847, AUC: 0.5842, Precision: 0.6252, Recall: 0.3957, MCC: 0.1822
Epoch 3/10
Train - Loss: 0.6628, Acc: 0.6178, F1: 0.5326, AUC: 0.6097, Precision: 0.6312, Recall: 0.4606, MCC: 0.2303
Val   - Loss: 0.6763, Acc: 0.5818, F1: 0.3438, AUC: 0.5743, Precision: 0.7399, Recall: 0.2239, MCC: 0.2090
Epoch 4/10
Train - Loss: 0.6519, Acc: 0.6397, F1: 0.5571, AUC: 0.6314, Precision: 0.6650, Recall: 0.4793, MCC: 0.2768
Val   - Loss: 0.6660, Acc: 0.5908, F1: 0.3629, AUC: 0.5834, Precision: 0.7614, Recall: 0.2382, MCC: 0.2315
Epoch 5/10
Train - Loss: 0.6403, Acc: 0.6577, F1: 0.5769, AUC: 0.6492, Precision: 0.6939, Recall: 0.

3 (IMR90)

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

4 (K692)

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

5 (NHEK)

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Virus

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")

Fungi

In [ ]:
for epoch in range(num_epochs):

    train_loss, train_acc, train_f1, train_auc, train_precision, train_recall, train_mcc = train(model, train_loader, criterion, optimizer, device)

    val_loss, val_acc, val_f1, val_auc, val_precision, val_recall, val_mcc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}, "
          f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, MCC: {val_mcc:.4f}")